In [50]:
import os
import urllib
import shutil
import azureml

from azureml.core import Experiment
from azureml.core import Workspace, Run

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.dnn import TensorFlow

In [51]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [45]:
!pip install image

ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


In [52]:
from azureml.core import Workspace, Dataset

from azureml.core.authentication import ServicePrincipalAuthentication
 
svc_pr_password = "1fY58u0dpP1Yg-i.A~rUp_iz04RxWUFSwv"
 
svc_pr = ServicePrincipalAuthentication(
    tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47",
    service_principal_id="8a3ddafe-6dd6-48af-867e-d745232a1833",
    service_principal_password="1fY58u0dpP1Yg-i.A~rUp_iz04RxWUFSwv")
 
ws = Workspace(
    subscription_id="c46a9435-c957-4e6c-a0f4-b9a597984773",
    resource_group="mlops",
    workspace_name="gputraining",
    auth=svc_pr
    )

In [53]:
ws

Workspace.create(name='gputraining', subscription_id='c46a9435-c957-4e6c-a0f4-b9a597984773', resource_group='mlops')

In [54]:
from azureml.core.datastore import Datastore

batchscore_blob = Datastore.register_azure_blob_container(ws, 
                      datastore_name="images_datastore", 
                      container_name="sampledata", 
                      account_name="pipelinedata", 
                      overwrite=True)

def_data_store = ws.get_default_datastore()

from azureml.core.dataset import Dataset
from azureml.pipeline.core import PipelineData

input_images = Dataset.File.from_files((batchscore_blob, "batchscoring/images/"))
label_ds = Dataset.File.from_files((batchscore_blob, "batchscoring/labels/"))
output_dir = PipelineData(name="scores", 
                          datastore=def_data_store, 
                          output_path_on_compute="batchscoring/results")

input_images = input_images.register(workspace = ws, name = "input_images")
label_ds = label_ds.register(workspace = ws, name = "label_ds")

In [55]:
from azureml.core.model import Model

model = Model(ws, 'tf-dnn-mnist')

In [56]:
model

Model(workspace=Workspace.create(name='gputraining', subscription_id='c46a9435-c957-4e6c-a0f4-b9a597984773', resource_group='mlops'), name=tf-dnn-mnist, id=tf-dnn-mnist:5, version=5, tags={}, properties={})

In [57]:
print(Model.get_model_path('tf-dnn-mnist',5,ws))

azureml-models/tf-dnn-mnist/5/model


In [58]:
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.exceptions import ComputeTargetException
compute_name = "gpucluster1"

# checks to see if compute target already exists in workspace, else create it
try:
    compute_target = ComputeTarget(workspace=ws, name=compute_name)
except ComputeTargetException:
    config = AmlCompute.provisioning_configuration(vm_size="STANDARD_NC6",
                                                   vm_priority="lowpriority", 
                                                   min_nodes=0, 
                                                   max_nodes=1)

    compute_target = ComputeTarget.create(workspace=ws, name=compute_name, provisioning_configuration=config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

In [59]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_GPU_IMAGE

cd = CondaDependencies.create(pip_packages=["tensorflow-gpu==1.15.2",
                                            "azureml-core", "azureml-dataprep[fuse]", "image"])
env = Environment(name="parallelenv")
env.python.conda_dependencies = cd
env.docker.base_image = DEFAULT_GPU_IMAGE

In [69]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_GPU_IMAGE

batch_conda_deps = CondaDependencies.create(pip_packages=["tensorflow==1.13.1", "pillow",
                                                          "azureml-core", "azureml-dataprep[pandas, fuse]"])

batch_env = Environment(name="batch_environment")
batch_env.python.conda_dependencies = batch_conda_deps
batch_env.docker.enabled = True
batch_env.docker.base_image = DEFAULT_GPU_IMAGE

In [71]:
from azureml.pipeline.steps import ParallelRunConfig

parallel_run_config = ParallelRunConfig(
    environment=env,
    entry_script="batchscore2.py",
    #source_directory="scripts",
    output_action="append_row",
    mini_batch_size="20",
    error_threshold=1,
    compute_target=compute_target,
    process_count_per_node=1,
    node_count=1
)

In [61]:
from azureml.pipeline.steps import ParallelRunStep
from datetime import datetime

parallel_step_name = "batchscoring-" + datetime.now().strftime("%Y%m%d%H%M")

label_config = label_ds.as_named_input("labels_input")

batch_score_step = ParallelRunStep(
    name=parallel_step_name,
    inputs=[input_images.as_named_input("input_images")],
    output=output_dir,
    arguments=["--model_name", "inception",
               "--labels_dir", label_config],
    side_inputs=[label_config],
    parallel_run_config=parallel_run_config,
    allow_reuse=False
)

In [67]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(workspace=ws, steps=[batch_score_step])
pipeline_run = Experiment(ws, 'batch_scoring').submit(pipeline)
pipeline_run.wait_for_completion(show_output=True)

Created step batchscoring-202007141306 [a2518915][dd875209-dad2-4f29-95ea-8a6a861f1b9c], (This step will run and generate new outputs)
Using data reference input_images_0 for StepId [6780b164][347a50ad-d876-4229-99c5-d977e6b60a3c], (Consumers of this data are eligible to reuse prior runs.)
Using data reference labels_input_0 for StepId [18fae126][1e2476f1-0ae1-4c13-92ab-e8449ea4e7dd], (Consumers of this data are eligible to reuse prior runs.)
Submitted PipelineRun f8f88a58-0a54-42fe-a33c-e957eb275901
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/batch_scoring/runs/f8f88a58-0a54-42fe-a33c-e957eb275901?wsid=/subscriptions/c46a9435-c957-4e6c-a0f4-b9a597984773/resourcegroups/mlops/workspaces/gputraining
PipelineRunId: f8f88a58-0a54-42fe-a33c-e957eb275901
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/batch_scoring/runs/f8f88a58-0a54-42fe-a33c-e957eb275901?wsid=/subscriptions/c46a9435-c957-4e6c-a0f4-b9a597984773/resourcegroups/mlops/workspa

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "User program failed with EntryScriptException: Job failed with entry script error No progress update in 270 seconds. Updated: False, total wait from last update: 275 seconds, remaining -5 seconds. 0/6 items processed. new_finished_tasks: 0. Detail: No mini batch has been processed.\nError 'The passed save_path is not a valid checkpoint: azureml-models/tf-dnn-mnist/5/model/tf-dnn-mnist.model' occurred 59 times.\nThe init() in the entry script had raised exception for 59 times. Please check agent log for more details. ",
        "detailsUri": "https://aka.ms/azureml-known-errors",
        "details": [],
        "debugInfo": {
            "type": "EntryScriptException",
            "message": "Job failed with entry script error No progress update in 270 seconds. Updated: False, total wait from last update: 275 seconds, remaining -5 seconds. 0/6 ite

In [68]:
import pandas as pd

batch_run = next(pipeline_run.get_children())
batch_output = batch_run.get_output_data("scores")
batch_output.download(local_path="inception_results")

KeyError: 'scores'

In [ ]:
import pandas as pd

batch_run = next(pipeline_run.get_children())
batch_output = batch_run.get_output_data("scores")
batch_output.download(local_path="inception_results")

for root, dirs, files in os.walk("inception_results"):
    for file in files:
        if file.endswith("parallel_run_step.txt"):
            result_file = os.path.join(root, file)

df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["Filename", "Prediction"]
print("Prediction has ", df.shape[0], " rows")
df.head(10)

In [ ]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Inception_v3_scoring", description="Batch scoring using Inception v3 model", version="1.0")

published_pipeline